In [1]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 12.0 MB/s 
     |████████████████████████████████| 3.4 MB 38.0 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=23b69142d08d91a1250b426cb20ef9ff3ad2f952d7a332f27b5dadbef66f50f9
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious


In [2]:
from pysat.formula import IDPool
from pysat.solvers import Cadical
from pysat.formula import CNF
cnf = CNF()
Atom = IDPool()

In [3]:
from google.colab import files
uploaded = files.upload()

Saving column_bin_5a_3p.csv to column_bin_5a_3p.csv


In [4]:
import pandas as pd
import io
 
arquivo = pd.read_csv(io.BytesIO(uploaded['column_bin_5a_3p.csv']))
arquivo

,PI <= 42.09,PI <= 70.62,PI <= 80.61,GS <= 37.89,GS <= 57.55,P
0,0,0,1,0,1,1
1,0,0,0,0,0,1
2,0,0,0,1,1,0


In [5]:
#Quantidade de pacientes do DataFrame
qtdPacientes=arquivo.shape[0]
print("Quantidade de Pacientes: ",qtdPacientes)

#Atributos do DataFrame
lista_atributos=arquivo.columns.tolist()
print("Atributos: ",lista_atributos)

#Quantidade de atributos do DataFrame
print("Quantidade de Atributos: ",arquivo.shape[1])

#Valoração
valoracao= [tuple(x) for x in arquivo.values]
print(valoracao)

#Quantidade de Regras
m=4

Quantidade de Pacientes:  3
Atributos:  ['PI <= 42.09', 'PI <= 70.62', 'PI <= 80.61', 'GS <= 37.89', 'GS <= 57.55', 'P']
Quantidade de Atributos:  6
[(0, 0, 1, 0, 1, 1), (0, 0, 0, 0, 0, 1), (0, 0, 0, 1, 1, 0)]


## **Restrição 1** 

In [30]:
def restricao1(m, lista_atributos):

    lista = []
    lista2 = []
    formula = []
    regra=0
    while True:
      if regra == m:
        break;
      for atributo in lista_atributos:
              if atributo != 'P':

                  lista=[]
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                  lista2.append(lista)
                  lista=[]

                  lista=[]
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                  lista2.append(lista)

                  lista=[]
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                  lista2.append(lista)

                  lista=[]
                  lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                  lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                  lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                  lista2.append(lista)
                  
      regra+=1

    return lista2

r1= restricao1(m, lista_atributos)
cnf.extend(r1)
print(r1)

[[-1, -2], [-1, -3], [-2, -3], [1, 2, 3], [-4, -5], [-4, -6], [-5, -6], [4, 5, 6], [-7, -8], [-7, -9], [-8, -9], [7, 8, 9], [-10, -11], [-10, -12], [-11, -12], [10, 11, 12], [-13, -14], [-13, -15], [-14, -15], [13, 14, 15], [-16, -17], [-16, -18], [-17, -18], [16, 17, 18], [-19, -20], [-19, -21], [-20, -21], [19, 20, 21], [-22, -23], [-22, -24], [-23, -24], [22, 23, 24], [-25, -26], [-25, -27], [-26, -27], [25, 26, 27], [-28, -29], [-28, -30], [-29, -30], [28, 29, 30], [-31, -32], [-31, -33], [-32, -33], [31, 32, 33], [-34, -35], [-34, -36], [-35, -36], [34, 35, 36], [-37, -38], [-37, -39], [-38, -39], [37, 38, 39], [-40, -41], [-40, -42], [-41, -42], [40, 41, 42], [-43, -44], [-43, -45], [-44, -45], [43, 44, 45], [-46, -47], [-46, -48], [-47, -48], [46, 47, 48], [-49, -50], [-49, -51], [-50, -51], [49, 50, 51], [-52, -53], [-52, -54], [-53, -54], [52, 53, 54], [-55, -56], [-55, -57], [-56, -57], [55, 56, 57], [-58, -59], [-58, -60], [-59, -60], [58, 59, 60]]


## **Restrição 2** 

In [31]:
def restricao2(m, lista_atributos):
    lista1 = []
    regra=0
    while regra < m:
        lista2 = []
        for atributo in range(0, len(lista_atributos) - 1):
          if atributo !='P':
            lista2.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
        lista1.append(lista2)
        regra+=1;
    return(lista1)

r2= restricao2(m, lista_atributos)
cnf.extend(r2)
print(r2)

[[-81, -82, -83, -84, -85], [-86, -87, -88, -89, -90], [-91, -92, -93, -94, -95], [-96, -97, -98, -99, -100]]


## **Restrição 3** 

In [39]:
def restricao3(m, lista_atributos, qtdPacientes, valoracao):
    lista2 = []
    j=0
    regra=0

    while True:
      if j == qtdPacientes:
          break
      if valoracao[j][-1] == 0: 
          while True:
            if regra == m:
              break;
            lista = []
            for atributo in lista_atributos:
                  if atributo != 'P':
                    if valoracao[j][lista_atributos.index(str(atributo))] == 1: 
                        lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n')) 
                    elif valoracao[j][lista_atributos.index(str(atributo))] == 0:
                        lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
            regra+=1
            lista2.append(lista)
      j+=1
    return lista2

r3= restricao3(m, lista_atributos, qtdPacientes, valoracao)
cnf.extend(r3)
print(r3)

[[-1, -4, -7, -11, -14], [-16, -19, -22, -26, -29], [-31, -34, -37, -41, -44], [-46, -49, -52, -56, -59]]


## **Restrição 4** 

In [38]:
def restricao4(m, qtdPacientes, lista_atributos, valoracao):
    lista = []
    lista2 = []

    for j in range(qtdPacientes):
        if valoracao[j][-1] == 1:
            for regra in range(m):     
                for atributo in lista_atributos:
                    lista = []
                    if atributo != 'P':
                        if valoracao[j][lista_atributos.index(str(atributo))] == 1:
                            lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                            lista.append(-Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
                        else:
                            lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                            lista.append(-Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
                        lista2.append(lista)

    return lista2

r4=restricao4(m ,qtdPacientes, lista_atributos, valoracao)
cnf.extend(r4)
print(r4)

[[-1, -73], [-4, -73], [-8, -73], [-10, -73], [-14, -73], [-16, -74], [-19, -74], [-23, -74], [-25, -74], [-29, -74], [-31, -75], [-34, -75], [-38, -75], [-40, -75], [-44, -75], [-46, -76], [-49, -76], [-53, -76], [-55, -76], [-59, -76], [-1, -77], [-4, -77], [-7, -77], [-10, -77], [-13, -77], [-16, -78], [-19, -78], [-22, -78], [-25, -78], [-28, -78], [-31, -79], [-34, -79], [-37, -79], [-40, -79], [-43, -79], [-46, -80], [-49, -80], [-52, -80], [-55, -80], [-58, -80]]


## **Restrição 5** 

In [37]:
def restricao5(m, qtdPacientes, valoracao, lista_atributos):
    lista2 = []
    for j in range(qtdPacientes):
        lista = []
        if valoracao[j][-1] == 1:
          for regra in range(m):
              lista.append(Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
          lista2.append(lista)
    return lista2
r5= restricao5(m, qtdPacientes, valoracao, lista_atributos)
cnf.extend(r5)
print(r5)

[[73, 74, 75, 76], [77, 78, 79, 80]]


## **Teste** 

In [36]:
print(cnf.clauses)
Formula = Cadical()
Formula.append_formula(cnf.clauses)

if Formula.solve():
    lista = Formula.get_model()
    for atomID in lista:
      if atomID < 0:
        print("\nAtom.id:", atomID, '=', 'Not (', Atom.obj(atomID*-1), end = '), ')
      else:
        print("\nAtom.id:", atomID, '=', Atom.obj(atomID),  end = ', ')
else:
    print("Não foi achada uma valoração")



[[1, -2, -3], [-1, 2, -3], [-1, -2, 3], [4, -5, -6], [-4, 5, -6], [-4, -5, 6], [7, -8, -9], [-7, 8, -9], [-7, -8, 9], [10, -11, -12], [-10, 11, -12], [-10, -11, 12], [13, -14, -15], [-13, 14, -15], [-13, -14, 15], [16, -17, -18], [-16, 17, -18], [-16, -17, 18], [19, -20, -21], [-19, 20, -21], [-19, -20, 21], [22, -23, -24], [-22, 23, -24], [-22, -23, 24], [25, -26, -27], [-25, 26, -27], [-25, -26, 27], [28, -29, -30], [-28, 29, -30], [-28, -29, 30], [31, -32, -33], [-31, 32, -33], [-31, -32, 33], [34, -35, -36], [-34, 35, -36], [-34, -35, 36], [37, -38, -39], [-37, 38, -39], [-37, -38, 39], [40, -41, -42], [-40, 41, -42], [-40, -41, 42], [43, -44, -45], [-43, 44, -45], [-43, -44, 45], [46, -47, -48], [-46, 47, -48], [-46, -47, 48], [49, -50, -51], [-49, 50, -51], [-49, -50, 51], [52, -53, -54], [-52, 53, -54], [-52, -53, 54], [55, -56, -57], [-55, 56, -57], [-55, -56, 57], [58, -59, -60], [-58, 59, -60], [-58, -59, 60], [-61, -61, -61, -61, -61], [-62, -62, -62, -62, -62], [-63, -63, -